In [1]:
import pandas as pd

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import requests
import time
from selenium.webdriver.chrome.service import Service
from collections import defaultdict
from selenium.webdriver.common.by import By

In [3]:
##Options for chromedriver, when using selenium, does not matter when using Collab since it acts more like a remote machine
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

In [4]:
#driver = webdriver.Chrome('chromedriver',options=chrome_options,executable_path='/path/to/chromedriver')
#driver = webdriver.Chrome(executable_path='/path/to/chromedriver', options=chrome_options)
service = Service('/path/to/chromedriver')

driver = webdriver.Chrome(service=service, options=chrome_options)


In [5]:
cricbuzz_highlights_url='https://www.cricbuzz.com/cricket-full-commentary/66169/gt-vs-csk-1st-match-indian-premier-league-2023'

In [6]:
#https://www.cricbuzz.com/cricket-series/5945/indian-premier-league-2023

In [7]:
driver.get(cricbuzz_highlights_url)

In [8]:
cricbuzz_soup= BeautifulSoup(driver.page_source, 'html.parser')

In [9]:
div_tag=cricbuzz_soup.find_all('div',{'class':"cb-col cb-col-100 ng-scope"})
#cb-com-ln ng-binding ng-scope cb-col cb-col-90

In [10]:
#class="cb-col cb-col-100 ng-scope"
#class="cb-col cb-col-100 ng-scope"
#class="cb-col cb-col-100 ng-scope"
#class="cb-com-ln ng-binding ng-scope cb-col cb-col-90"
#class="cb-col cb-col-20" match box
#class=cb-nav-subhdr cb-font-12  header 
#class="cb-billing-plans-text cb-team-lft-item" team names

In [11]:
#cricbuzz_soup.find_all('p',{'class':'cb-com-ln ng-binding ng-scope cb-col cb-col-90'})

In [12]:
match_metadata_values=[c.next_element.next_element.next_element.text.replace('\xa0','') for c in cricbuzz_soup.find_all('span',{'class':'text-bold'})]

match_metadata_values



['Indian Premier League 2023', 'Narendra Modi Stadium,Ahmedabad', 'Mar 31,']

In [13]:
year=match_metadata_values[0].split()[-1]
series="IPL"
venue=match_metadata_values[1]
date=match_metadata_values[2]+year

In [14]:
def match_no():
    team_names=cricbuzz_soup.find('div',{'class':"cb-billing-plans-text cb-team-lft-item"})
    teams=team_names.text.split(',')[1]
    return teams.strip()[0:3]

In [15]:
def short_name(team):
    short_name={'Chennai Super Kings':'CSK',
  'Mumbai Indians':'MI',
   'Gujarat Titans':'GT',
   'Kolkata Knight Riders':'KKR',
   'Punjab Kings':'PBKS',
   'SunRisers Hyderabad':'SRH',
   'Rajasthan Royals':'RR',
   'Lucknow Super Giants':'LSG',
   'Delhi Capitals':'DC',
   'Royal Challengers Bangalore':'RCB'
  }
    return short_name[team]

In [16]:
#cb-com-ln ng-binding ng-scope cb-col cb-col-90
def find_toss():
    toss=cricbuzz_soup.find_all('p',{'class':"cb-com-ln ng-binding ng-scope cb-col cb-col-90"})
    for i in toss:
        if "won the toss" in i.get_text().lower():
            return i.get_text()


In [17]:
def team_names():
    team_names=cricbuzz_soup.find('div',{'class':"cb-billing-plans-text cb-team-lft-item"})
    teams=team_names.text.split(',')[0]
    teams=teams.split('vs')
    teams=[team.strip() for team in teams]
    return teams

In [18]:
def match_info():
    match_details=cricbuzz_soup.find('div',{'class':"cb-col cb-col-20"})
    match_info=""
    for i in match_details:
        match_info+=i.get_text()
    return match_info    


In [19]:
def find_team_subs(team):
    preview_obj=cricbuzz_soup.find_all('p',{'class':'cb-com-ln ng-binding ng-scope cb-col cb-col-90'})
    for i in preview_obj:
        if 'subs' in i.text.lower() and (team in i.text or short_name(team) in i.text):
            return i.get_text().split(',')


In [20]:
def find_team_playing11(team):
    preview_obj=cricbuzz_soup.find_all('p',{'class':'cb-com-ln ng-binding ng-scope cb-col cb-col-90'})
    for i in preview_obj:
        if '(Playing XI)'.casefold() in i.text.casefold() and (team in i.text or short_name(team) in i.text):
            return i.get_text().split(',')

In [21]:
df=pd.DataFrame(columns=['match_no','series','season','team_a','team_b','team_a_11','team_b_11','team_a_subs','team_b_subs','venue','date','toss'])

In [22]:
match =match_no()
df.at[1,'match_no']=match

In [23]:
team_a,team_b=team_names()

In [24]:
df.at[1,'team_a']=team_a
df.at[1,'team_b']=team_b

In [25]:
df.at[1,'team_a_11']=find_team_playing11(team_a)
df.at[1,'team_b_11']=find_team_playing11(team_b)
df.at[1,'team_a_subs']=find_team_subs(team_a)
df.at[1,'team_b_subs']=find_team_subs(team_b)
df.at[1,'series']=series
df.at[1,'season']=year
df.at[1,'venue']=venue
df.at[1,'date']=date
df.at[1,'toss']=find_toss()

In [26]:
display(df)

,match_no,series,season,team_a,team_b,team_a_11,team_b_11,team_a_subs,team_b_subs,venue,date,toss
1,1st,IPL,2023,Gujarat Titans,Chennai Super Kings,[Gujarat Titans (Playing XI): Wriddhiman Saha(...,[Chennai Super Kings (Playing XI): Devon Conwa...,"[Gujarat Titans Subs: B Sai Sudharsan, Jayant...","[Chennai Super Kings Subs: Tushar Deshpande, ...","Narendra Modi Stadium,Ahmedabad","Mar 31,2023",Gujarat Titans have won the toss and have opte...


In [27]:
df.loc[1,'team_a_11']

['Gujarat Titans (Playing XI): Wriddhiman Saha(w)',
 ' Shubman Gill',
 ' Kane Williamson',
 ' Hardik Pandya(c)',
 ' Vijay Shankar',
 ' Rahul Tewatia',
 ' Rashid Khan',
 ' Mohammed Shami',
 ' Joshua Little',
 ' Yash Dayal',
 ' Alzarri Joseph']

In [28]:
link_text=[]
div_tag=cricbuzz_soup.find_all('div',{"class":"cb-hig-pil ng-scope"})
for a_tag in div_tag:
    link_text.append(a_tag.a.string)
link_text=link_text[1:]
link_text

['CSK Inns', 'GT Inns']

In [29]:
cricbuzz_page_soup=[]
##Iterate through the link textss
for l in link_text:
        ##Click the innings text
        
        try:
            loadMoreButton=driver.find_element(By.LINK_TEXT,l)
            loadMoreButton.click()
            time.sleep(5)  #must
            ##Collect the soup of the existing view that has been clicked by the driver element
            cricbuzz_soup_inner=BeautifulSoup(driver.page_source, 'html.parser')
            ##Append each of the page contents to a list
            cricbuzz_page_soup.append(cricbuzz_soup_inner)
            print(l,' Analysed')
        except:
            print('success')

##Scraping has been completed
print ("Full scraping of key events complete...")
##Stop the driver element
driver.quit()

CSK Inns  Analysed
GT Inns  Analysed
Full scraping of key events complete...


In [30]:
match_commentary_df=pd.DataFrame()
for inn_num,cricbuzz_highlights_soup in enumerate(cricbuzz_page_soup):
    cricbuzz_innings_soup=cricbuzz_highlights_soup.find_all('div',{'class':'cb-col cb-col-8 text-bold ng-scope'})
    ball_overs=[]
    ball_commentary=[]
    for cinn_soup in cricbuzz_innings_soup:
        ball_overs.append(cinn_soup.text)
        ball_commentary.append(str(cinn_soup.find_next('p').text))
        #ball_commentry.append()
    innings_commentary_df=pd.DataFrame({'ball':ball_overs,'Commentary Text':ball_commentary})
    innings_commentary_df['ball']=innings_commentary_df['ball']
    innings_commentary_df['innings']=link_text[inn_num]
    match_commentary_df=pd.concat([match_commentary_df,innings_commentary_df])

In [31]:
match_commentary_df.reset_index(inplace=True,drop=True)
match_commentary_df

,ball,Commentary Text,innings
0,19.6,"Joshua Little to Dhoni, 1 run, CSK finish wit...",CSK Inns
1,19.5,"Joshua Little to Dhoni, no run, slower one ou...",CSK Inns
2,19.4,"Joshua Little to Dhoni, FOUR, and now Hardik ...",CSK Inns
3,19.3,"Joshua Little to Dhoni, SIX, and MSD makes su...",CSK Inns
4,19.2,"Joshua Little to Santner, leg byes, 1 run, fi...",CSK Inns
...,...,...,...
238,0.5,"Chahar to Shubman Gill, 2 runs, Gill is up an...",GT Inns
239,0.4,"Chahar to Saha, 1 run, on a length outside of...",GT Inns
240,0.3,"Chahar to Saha, no run, sharp inswinger on mi...",GT Inns
241,0.2,"Chahar to Saha, no run, curving away from the...",GT Inns


In [32]:
match_commentary_df

,ball,Commentary Text,innings
0,19.6,"Joshua Little to Dhoni, 1 run, CSK finish wit...",CSK Inns
1,19.5,"Joshua Little to Dhoni, no run, slower one ou...",CSK Inns
2,19.4,"Joshua Little to Dhoni, FOUR, and now Hardik ...",CSK Inns
3,19.3,"Joshua Little to Dhoni, SIX, and MSD makes su...",CSK Inns
4,19.2,"Joshua Little to Santner, leg byes, 1 run, fi...",CSK Inns
...,...,...,...
238,0.5,"Chahar to Shubman Gill, 2 runs, Gill is up an...",GT Inns
239,0.4,"Chahar to Saha, 1 run, on a length outside of...",GT Inns
240,0.3,"Chahar to Saha, no run, sharp inswinger on mi...",GT Inns
241,0.2,"Chahar to Saha, no run, curving away from the...",GT Inns


In [33]:
df

,match_no,series,season,team_a,team_b,team_a_11,team_b_11,team_a_subs,team_b_subs,venue,date,toss
1,1st,IPL,2023,Gujarat Titans,Chennai Super Kings,[Gujarat Titans (Playing XI): Wriddhiman Saha(...,[Chennai Super Kings (Playing XI): Devon Conwa...,"[Gujarat Titans Subs: B Sai Sudharsan, Jayant...","[Chennai Super Kings Subs: Tushar Deshpande, ...","Narendra Modi Stadium,Ahmedabad","Mar 31,2023",Gujarat Titans have won the toss and have opte...


In [34]:
match_commentary_df

,ball,Commentary Text,innings
0,19.6,"Joshua Little to Dhoni, 1 run, CSK finish wit...",CSK Inns
1,19.5,"Joshua Little to Dhoni, no run, slower one ou...",CSK Inns
2,19.4,"Joshua Little to Dhoni, FOUR, and now Hardik ...",CSK Inns
3,19.3,"Joshua Little to Dhoni, SIX, and MSD makes su...",CSK Inns
4,19.2,"Joshua Little to Santner, leg byes, 1 run, fi...",CSK Inns
...,...,...,...
238,0.5,"Chahar to Shubman Gill, 2 runs, Gill is up an...",GT Inns
239,0.4,"Chahar to Saha, 1 run, on a length outside of...",GT Inns
240,0.3,"Chahar to Saha, no run, sharp inswinger on mi...",GT Inns
241,0.2,"Chahar to Saha, no run, curving away from the...",GT Inns


In [43]:
final

,ball,Commentary Text,innings,match_no,series,season,team_a,team_b,team_a_11,team_b_11,team_a_subs,team_b_subs,venue,date,toss
0,19.6,"Joshua Little to Dhoni, 1 run, CSK finish wit...",CSK Inns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19.5,"Joshua Little to Dhoni, no run, slower one ou...",CSK Inns,1st,IPL,2023,Gujarat Titans,Chennai Super Kings,[Gujarat Titans (Playing XI): Wriddhiman Saha(...,[Chennai Super Kings (Playing XI): Devon Conwa...,"[Gujarat Titans Subs: B Sai Sudharsan, Jayant...","[Chennai Super Kings Subs: Tushar Deshpande, ...","Narendra Modi Stadium,Ahmedabad","Mar 31,2023",Gujarat Titans have won the toss and have opte...
2,19.4,"Joshua Little to Dhoni, FOUR, and now Hardik ...",CSK Inns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19.3,"Joshua Little to Dhoni, SIX, and MSD makes su...",CSK Inns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19.2,"Joshua Little to Santner, leg byes, 1 run, fi...",CSK Inns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.5,"Chahar to Shubman Gill, 2 runs, Gill is up an...",GT Inns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,0.4,"Chahar to Saha, 1 run, on a length outside of...",GT Inns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,0.3,"Chahar to Saha, no run, sharp inswinger on mi...",GT Inns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,0.2,"Chahar to Saha, no run, curving away from the...",GT Inns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
final=pd.concat([match_commentary_df,df],axis=1)

In [58]:
final

,ball,Commentary Text,innings,match_no,series,season,team_a,team_b,team_a_11,team_b_11,team_a_subs,team_b_subs,venue,date,toss
0,19.6,"Joshua Little to Dhoni, 1 run, CSK finish wit...",CSK Inns,1st,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gujarat Titans have won the toss and have opte...
1,19.5,"Joshua Little to Dhoni, no run, slower one ou...",CSK Inns,1st,IPL,2023,Gujarat Titans,Chennai Super Kings,[Gujarat Titans (Playing XI): Wriddhiman Saha(...,[Chennai Super Kings (Playing XI): Devon Conwa...,"[Gujarat Titans Subs: B Sai Sudharsan, Jayant...","[Chennai Super Kings Subs: Tushar Deshpande, ...","Narendra Modi Stadium,Ahmedabad","Mar 31,2023",Gujarat Titans have won the toss and have opte...
2,19.4,"Joshua Little to Dhoni, FOUR, and now Hardik ...",CSK Inns,1st,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gujarat Titans have won the toss and have opte...
3,19.3,"Joshua Little to Dhoni, SIX, and MSD makes su...",CSK Inns,1st,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gujarat Titans have won the toss and have opte...
4,19.2,"Joshua Little to Santner, leg byes, 1 run, fi...",CSK Inns,1st,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gujarat Titans have won the toss and have opte...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.5,"Chahar to Shubman Gill, 2 runs, Gill is up an...",GT Inns,1st,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gujarat Titans have won the toss and have opte...
239,0.4,"Chahar to Saha, 1 run, on a length outside of...",GT Inns,1st,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gujarat Titans have won the toss and have opte...
240,0.3,"Chahar to Saha, no run, sharp inswinger on mi...",GT Inns,1st,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gujarat Titans have won the toss and have opte...
241,0.2,"Chahar to Saha, no run, curving away from the...",GT Inns,1st,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gujarat Titans have won the toss and have opte...
